In [2]:
from typing import List
from pydantic import BaseModel


class City(BaseModel):
    # 城市名稱
    name: str
    # 國家
    country: str
    # 人口
    population: int


class CitiesData(BaseModel):
    # 城市列表
    cities: List[City]

In [5]:
json_schema = CitiesData.schema_json(indent=2)
json_schema

'{\n  "$defs": {\n    "City": {\n      "properties": {\n        "name": {\n          "title": "Name",\n          "type": "string"\n        },\n        "country": {\n          "title": "Country",\n          "type": "string"\n        },\n        "population": {\n          "title": "Population",\n          "type": "integer"\n        }\n      },\n      "required": [\n        "name",\n        "country",\n        "population"\n      ],\n      "title": "City",\n      "type": "object"\n    }\n  },\n  "properties": {\n    "cities": {\n      "items": {\n        "$ref": "#/$defs/City"\n      },\n      "title": "Cities",\n      "type": "array"\n    }\n  },\n  "required": [\n    "cities"\n  ],\n  "title": "CitiesData",\n  "type": "object"\n}'

In [6]:
import json
import random
import pydantic
from pydantic import ValidationError
from typing import Optional, List
from colorama import Fore
from haystack import component

# 定義組件的輸入參數
@component
class OutputValidator:
    def __init__(self, pydantic_model: pydantic.BaseModel):
        self.pydantic_model = pydantic_model  # 保存 Pydantic 模型
        self.iteration_counter = 0  # 初始化循環計數器

    # 定義組件的輸出
    @component.output_types(valid_replies=List[str], invalid_replies=Optional[List[str]], error_message=Optional[str])
    def run(self, replies: List[str]):
        self.iteration_counter += 1  # 增加循環計數器

        ## 嘗試解析 LLM 的回覆 ##
        # 如果 LLM 的回覆是一個有效的對象，返回 `"valid_replies"`
        try:
            output_dict = json.loads(replies[0])  # 解析回覆為字典
            self.pydantic_model.parse_obj(output_dict)  # 使用 Pydantic 模型進行驗證
            print(
                Fore.GREEN
                + f"OutputValidator at Iteration {self.iteration_counter}: Valid JSON from LLM - No need for looping: {replies[0]}"
            )
            return {"valid_replies": replies}

        # 如果 LLM 的回覆損壞或無效，返回 "invalid_replies" 和 "error_message" 以便 LLM 重試
        except (ValueError, ValidationError) as e:
            print(
                Fore.RED
                + f"OutputValidator at Iteration {self.iteration_counter}: Invalid JSON from LLM - Let's try again.\n"
                f"Output from LLM:\n {replies[0]} \n"
                f"Error from OutputValidator: {e}"
            )
            return {"invalid_replies": replies, "error_message": str(e)}

/Users/samhsiao/Documents/PythonVenv/envHaystack/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [10]:
output_validator = OutputValidator(pydantic_model=CitiesData)
output_validator

Inputs:
  - replies: List[str]
Outputs:
  - valid_replies: List[str]
  - invalid_replies: Optional[List[str]]
  - error_message: Optional[str]